## Importing Library's:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sna
import matplotlib.pyplot as plt

##  Extracting Data from Data Source(SQL_Database):

In [2]:
import pandas as pd
import pyodbc

# Database connection parameters
server = 'OMKAR'
database = 'master'
username = 'admin'
password = '123456789'

# Create a connection string
conn_str = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Create a database connection
conn = pyodbc.connect(conn_str)

# SQL query to fetch data (replace 'your_table' with the actual table name)
query = 'SELECT * FROM superstore_raw_data'

# Use Pandas to read data into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
df.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_22112\2599352051.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


## Exporting Data into Bronze File:

In [3]:
df.to_parquet("C:\Data_Storage\Bronze\Superstore_raw_data.parquet",index=False)

## Preliminary Information Of Data:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65535 entries, 0 to 65534
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         65535 non-null  int64         
 1   Order ID       65535 non-null  object        
 2   Order Date     65535 non-null  datetime64[ns]
 3   Ship Date      65535 non-null  datetime64[ns]
 4   Ship Mode      65535 non-null  object        
 5   Customer ID    65535 non-null  object        
 6   Customer Name  65535 non-null  object        
 7   Segment        65535 non-null  object        
 8   Country        65535 non-null  object        
 9   City           65535 non-null  object        
 10  State          65535 non-null  object        
 11  Postal Code    65478 non-null  float64       
 12  Region         65535 non-null  object        
 13  Product ID     65535 non-null  object        
 14  Category       65535 non-null  object        
 15  Sub-Category   6553

In [5]:
print(f'The table contains {df.shape[0]} rows and {df.shape[1]} columns.')

The table contains 65535 rows and 21 columns.


## Data Cleaning :

### 1. Treatment of Duplicate values :

In [6]:
print(f'The table contains {df.duplicated().sum()} duplicate rows')   ### table contains 55515 duplicate rows so, it remove from dataset.

The table contains 55515 duplicate rows


In [7]:
df.drop_duplicates(inplace=True)

In [8]:
print(f'The table contains {df.duplicated().sum()} duplicate rows after treatment of duplicate values')

The table contains 0 duplicate rows after treatment of duplicate values


### 2. Treatment of Missing values :

In [9]:
print(f'The table contains {df.isna().sum().sum()} missing values')

The table contains 11 missing values


In [10]:
df.isna().sum()      ## "Postal Code" column contain 11 missing values, so, these values are replaced by mode.

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64

In [11]:
df['Postal Code'].fillna(df['Postal Code'].mode(),inplace=True)

### Check data:

In [12]:
for i in df:
    print(df[i].value_counts())   ## Noice in data is not present

Row ID
3591    2
5760    2
9388    2
9387    2
8799    2
       ..
3337    1
3338    1
3339    1
3340    1
9994    1
Name: count, Length: 9994, dtype: int64
Order ID
CA-2017-100111    14
CA-2017-157987    12
CA-2016-165330    11
US-2016-108504    11
CA-2015-131338    10
                  ..
CA-2017-109778     1
CA-2017-148929     1
CA-2016-118332     1
CA-2016-133669     1
CA-2017-119914     1
Name: count, Length: 5009, dtype: int64
Order Date
2016-09-05    38
2017-09-02    36
2016-11-10    36
2017-12-02    34
2017-12-01    34
              ..
2016-02-25     1
2014-01-14     1
2016-10-25     1
2014-02-21     1
2014-01-21     1
Name: count, Length: 1237, dtype: int64
Ship Date
2015-12-16    35
2017-09-26    34
2017-11-21    32
2017-12-06    32
2017-09-06    30
              ..
2014-02-12     1
2014-03-28     1
2016-02-18     1
2017-03-19     1
2016-10-05     1
Name: count, Length: 1334, dtype: int64
Ship Mode
Standard Class    5989
Second Class      1948
First Class       1539
Same Day 

##  Exporting Data into Silver File:

In [13]:
df.to_parquet("C:\Data_Storage\Silver\Superstore_cleaned_data.parquet",index=False)

## Data transformation:

####  Days to Ship: Calculating the number of days it took to ship each order.

In [14]:
df['Days to Ship'] = (df['Ship Date'] - df['Order Date']).dt.days

####  Revenue per Order

In [15]:
df['Revenue per Order'] = df['Sales'] / df['Quantity']

#### Discounted Amount

In [16]:
 df['Discounted Amount'] = df['Sales'] * df['Discount']

####  Profit Margin

In [17]:
df['Profit Margin'] = (df['Profit'] / df['Sales']) * 100

#### Seasonal Analysis

In [18]:
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year

####  Customer Tenure:
Calculating the tenure of each customer by finding the difference between the first and last order dates for each customer.

In [19]:
df['Customer Tenure'] = (df.groupby('Customer ID')['Order Date'].transform('max') - df.groupby('Customer ID')['Order Date'].transform('min')).dt.days

####  Cumulative Sales and Profit:
Calculating the cumulative sales and profit over time to observe the overall trend.

In [20]:
df['Cumulative Sales'] = df.groupby('Order Date')['Sales'].transform('cumsum')
df['Cumulative Profit'] = df.groupby('Order Date')['Profit'].transform('cumsum')

######  Order Size Category:
Categorizing orders based on the quantity of products ordered (e.g., Small, Medium, Large). 

In [21]:
df['Order Size Category'] = pd.cut(df['Quantity'], bins=[0, 5, 10, float('inf')], labels=['Small', 'Medium', 'Large'])

In [22]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Days to Ship,Revenue per Order,Discounted Amount,Profit Margin,Month,Year,Customer Tenure,Cumulative Sales,Cumulative Profit,Order Size Category
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,3,130.9800,0.000000,16.00,11,2016,469,261.9600,41.9136,Small
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,3,243.9800,0.000000,30.00,11,2016,469,993.9000,261.4956,Small
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,4,7.3100,0.000000,47.00,6,2016,547,14.6200,6.8714,Small
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,7,191.5155,430.909875,-40.00,10,2015,782,957.5775,-383.0310,Small
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,7,11.1840,4.473600,11.25,10,2015,782,979.9455,-380.5146,Small


## Preliminary Information of Data after Data Cleaning & Data Transformation :

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10020 entries, 0 to 19783
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Row ID               10020 non-null  int64         
 1   Order ID             10020 non-null  object        
 2   Order Date           10020 non-null  datetime64[ns]
 3   Ship Date            10020 non-null  datetime64[ns]
 4   Ship Mode            10020 non-null  object        
 5   Customer ID          10020 non-null  object        
 6   Customer Name        10020 non-null  object        
 7   Segment              10020 non-null  object        
 8   Country              10020 non-null  object        
 9   City                 10020 non-null  object        
 10  State                10020 non-null  object        
 11  Postal Code          10009 non-null  float64       
 12  Region               10020 non-null  object        
 13  Product ID           10020 non-null 

In [24]:
print(f'The table contains {df.shape[0]} rows and {df.shape[1]} columns.')

The table contains 10020 rows and 31 columns.


##  Exporting Data into Gold File:

In [25]:
df.to_parquet("C:\Data_Storage\Gold\Superstore_buziness_data.parquet",index=False)